In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [51]:
from fastai import *


from pathlib import Path
import json
import collections
from PIL import ImageDraw, ImageFont
from matplotlib import patches, patheffects

import numpy as np

In [6]:
PATH = Path('../../../data/pascal/')
list(PATH.iterdir())

[PosixPath('../../../data/pascal/pascal_val2007.json'),
 PosixPath('../../../data/pascal/VOCdevkit'),
 PosixPath('../../../data/pascal/pascal_train2012.json'),
 PosixPath('../../../data/pascal/pascal_test2007.json'),
 PosixPath('../../../data/pascal/VOCtrainval_06-Nov-2007.tar'),
 PosixPath('../../../data/pascal/PASCAL_VOC.zip'),
 PosixPath('../../../data/pascal/pascal_train2007.json'),
 PosixPath('../../../data/pascal/pascal_val2012.json')]

In [10]:
trn_j = json.load((PATH/'pascal_train2007.json').open())
trn_j.keys()

dict_keys(['images', 'type', 'annotations', 'categories'])

In [8]:
IMAGES,ANNOTATIONS,CATEGORIES = ['images', 'annotations', 'categories']
trn_j[IMAGES][:5]

[{'file_name': '000012.jpg', 'height': 333, 'width': 500, 'id': 12},
 {'file_name': '000017.jpg', 'height': 364, 'width': 480, 'id': 17},
 {'file_name': '000023.jpg', 'height': 500, 'width': 334, 'id': 23},
 {'file_name': '000026.jpg', 'height': 333, 'width': 500, 'id': 26},
 {'file_name': '000032.jpg', 'height': 281, 'width': 500, 'id': 32}]

In [11]:
trn_j[ANNOTATIONS][:2]

[{'segmentation': [[155, 96, 155, 270, 351, 270, 351, 96]],
  'area': 34104,
  'iscrowd': 0,
  'image_id': 12,
  'bbox': [155, 96, 196, 174],
  'category_id': 7,
  'id': 1,
  'ignore': 0},
 {'segmentation': [[184, 61, 184, 199, 279, 199, 279, 61]],
  'area': 13110,
  'iscrowd': 0,
  'image_id': 17,
  'bbox': [184, 61, 95, 138],
  'category_id': 15,
  'id': 2,
  'ignore': 0}]

In [12]:
trn_j[CATEGORIES][:4]

[{'supercategory': 'none', 'id': 1, 'name': 'aeroplane'},
 {'supercategory': 'none', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'none', 'id': 3, 'name': 'bird'},
 {'supercategory': 'none', 'id': 4, 'name': 'boat'}]

In [13]:
FILE_NAME,ID,IMG_ID,CAT_ID,BBOX = 'file_name','id','image_id','category_id','bbox'

cats = {o[ID]:o['name'] for o in trn_j[CATEGORIES]}
trn_fns = {o[ID]:o[FILE_NAME] for o in trn_j[IMAGES]}
trn_ids = [o[ID] for o in trn_j[IMAGES]]

In [14]:
cats

{1: 'aeroplane',
 2: 'bicycle',
 3: 'bird',
 4: 'boat',
 5: 'bottle',
 6: 'bus',
 7: 'car',
 8: 'cat',
 9: 'chair',
 10: 'cow',
 11: 'diningtable',
 12: 'dog',
 13: 'horse',
 14: 'motorbike',
 15: 'person',
 16: 'pottedplant',
 17: 'sheep',
 18: 'sofa',
 19: 'train',
 20: 'tvmonitor'}

In [17]:
list((PATH/'VOCdevkit'/'VOC2007').iterdir())

[PosixPath('../../../data/pascal/VOCdevkit/VOC2007/ImageSets'),
 PosixPath('../../../data/pascal/VOCdevkit/VOC2007/SegmentationClass'),
 PosixPath('../../../data/pascal/VOCdevkit/VOC2007/SegmentationObject'),
 PosixPath('../../../data/pascal/VOCdevkit/VOC2007/Annotations'),
 PosixPath('../../../data/pascal/VOCdevkit/VOC2007/JPEGImages')]

In [18]:
JPEGS = 'VOCdevkit/VOC2007/JPEGImages'

In [19]:
IMG_PATH = PATH/JPEGS
list(IMG_PATH.iterdir())[:5]

[PosixPath('../../../data/pascal/VOCdevkit/VOC2007/JPEGImages/003301.jpg'),
 PosixPath('../../../data/pascal/VOCdevkit/VOC2007/JPEGImages/006279.jpg'),
 PosixPath('../../../data/pascal/VOCdevkit/VOC2007/JPEGImages/008254.jpg'),
 PosixPath('../../../data/pascal/VOCdevkit/VOC2007/JPEGImages/007167.jpg'),
 PosixPath('../../../data/pascal/VOCdevkit/VOC2007/JPEGImages/000608.jpg')]

In [20]:
im0_d = trn_j[IMAGES][0]
im0_d[FILE_NAME],im0_d[ID]

('000012.jpg', 12)

In [34]:
def hw_bb(bb): return np.array([bb[1], bb[0], bb[3]+bb[1]-1, bb[2]+bb[0]-1])

trn_anno = collections.defaultdict(lambda:[])
for o in trn_j[ANNOTATIONS]:
    if not o['ignore']:
        bb = o[BBOX]
        bb = hw_bb(bb)
        trn_anno[o[IMG_ID]].append((bb,o[CAT_ID]))
        
len(trn_anno)




2501

In [35]:
im_a = trn_anno[im0_d[ID]]; im_a

[(array([ 96, 155, 269, 350]), 7)]

In [36]:
im0_a = im_a[0]; im0_a

(array([ 96, 155, 269, 350]), 7)

In [37]:
cats[7]

'car'

In [38]:
trn_anno[17]

[(array([ 61, 184, 198, 278]), 15), (array([ 77,  89, 335, 402]), 13)]

In [39]:
bb_voc = [155, 96, 196, 174]
bb_fastai = hw_bb(bb_voc)

In [40]:
def bb_hw(a): return np.array([a[1], a[0], a[3]-a[1]+1, a[2]-a[0]+1])

In [41]:
f'expected: {bb_voc}, actual: {bb_hw(bb_fastai)}'

'expected: [155, 96, 196, 174], actual: [155  96 196 174]'

In [52]:
im = open_image(IMG_PATH/im0_d[FILE_NAME])

NameError: name 'open_image' is not defined

In [49]:
im

PosixPath('../../../data/pascal/VOCdevkit/VOC2007/JPEGImages/000012.jpg')